# snntorch - Tutorial 2

## Imports

In [1]:
# Spiking Neural Network
import snntorch as snn
from snntorch import utils
from snntorch import spikegen

# Torch
import torch
from torch.utils.data import DataLoader

# Torchvision
from torchvision import datasets, transforms

# Visualization
import matplotlib.pyplot as plt
import snntorch.spikeplot as splt
from IPython.display import HTML

- *Hodgkin-Huxley Neuron Models*: Biophysically accurate but complex.
- *Artificial Neuron Model*: Inputs are mulitplied by their corresponding weights and passed through an activation function. Standard for traditional deep learning.
- *Leaky Integrate-and-Fire Neuron Models*: Somewhere in the middle. Takes the sum of weighted inputs, but instead of passing it through an activation function, it will integrate the input over time with a leakage. If the integrated value exceeds a threshold then the neuron will fire.

In [ ]:
def leaky_integrate_neuron(U, time_step=1e-3, I=0, R=5e7, C=1e-10):
    tau = R*C
    U = U + (time_step/tau)*(-U + I*R)
    return U

num_steps = 100
U = 0.9
U_trace = []  # keeps a record of U for plotting

for step in range(num_steps):
    U_trace.append(U)
    U = leaky_integrate_neuron(U)  # solve next step of U

fig, ax = plt.subplots()
ax.plot(U_trace)
ax.set_title('Leaky Integrate-and-Fire Neuron')

## Lapicque's LIF Neuron Model

We store the neuron in `lif1`.

Modelled as an RC circuit.

In [ ]:
time_step = 1e-3
R = 5
C = 1e-3

# leaky integrate and fire neuron, tau=5e-3
lif1 = snn.Lapicque(R=R, C=C, time_step=time_step)

# Initialize membrane, input, and output
mem = torch.ones(1) * 0.9  # U=0.9 at t=0
cur_in = torch.zeros(num_steps, 1)  # I=0 for all t
spk_out = torch.zeros(1)  # initialize output spikes

# A list to store a recording of membrane potential
mem_rec = [mem]

# pass updated value of mem and cur_in[step]=0 at every time step
for step in range(num_steps):
    spk_out, mem = lif1(cur_in[step], mem)

    # Store recordings of membrane potential
    mem_rec.append(mem)

# convert the list of tensors into one tensor
mem_rec = torch.stack(mem_rec)

fig, ax = plt.subplots()
ax.plot(mem_rec)
ax.set_title("Lapicque's Neuron Model Without Stimulus")

In [ ]:
num_steps = 100
time_step = 1e-3
R = 5
C = 1e-3

# leaky integrate and fire neuron, tau=5e-3
lif1 = snn.Lapicque(R=R, C=C, time_step=time_step)

# Initialize membrane, input, and output
mem = torch.zeros(1)
cur_in = torch.cat((torch.zeros(10, 1), torch.ones(190, 1)*0.1), 0)  # I=0 for all t
spk_out = torch.zeros(1)  # initialize output spikes

# A list to store a recording of membrane potential
mem_rec = [mem]

# pass updated value of mem and cur_in[step]=0 at every time step
for step in range(num_steps):
    spk_out, mem = lif1(cur_in[step], mem)

    # Store recordings of membrane potential
    mem_rec.append(mem)

# convert the list of tensors into one tensor
mem_rec = torch.stack(mem_rec)

fig, axs = plt.subplots(2, 1)
axs[0].plot(mem_rec)
axs[1].plot(cur_in)
axs[0].set_title("Lapicque's Neuron Model with Stimulus")
axs[0].set_xlim(0, num_steps)
axs[1].set_xlim(0, num_steps)

## Lapicque: Firing

In [ ]:
def leaky_integrate_and_fire(mem, cur=0, threshold=1, time_step=1e-3, R=5.1, C=5e-3):
    tau_mem = R*C
    spk = (mem > threshold)
    mem = mem + (time_step/tau_mem)*(-mem + cur*R) - spk*threshold  # every time spk=1, subtract the threhsold
    return mem, spk

num_steps = 200

# Small step current input
cur_in = torch.cat((torch.zeros(10), torch.ones(190)*0.2), 0)
mem = torch.zeros(1)
mem_rec = []
spk_rec = []

# neuron simulation
for step in range(num_steps):
    mem, spk = leaky_integrate_and_fire(mem, cur_in[step])
    mem_rec.append(mem)
    spk_rec.append(spk)

# convert lists to tensors
mem_rec = torch.stack(mem_rec)
spk_rec = torch.stack(spk_rec)

fig, axs = plt.subplots(3, 1)
axs[0].plot(cur_in)
axs[1].plot(mem_rec)
axs[2].plot(spk_rec)